In [1]:
import Homework1

/Users/nickeldime/Code/LLM Class/CS4973/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Task 2

In [2]:
EASY = [
    {
        "question": "I ate 2 pears and 1 apple. How many fruit did I eat?",
        "answer": 3
    },
    {
        "question": "I had ten chocolates but ate one. How many remain?",
        "answer": 9
    }
]

Homework1.accuracy_zero_shot(EASY)

  0%|          | 0/5 [00:00<?, ?it/s]

[1.0, 1.0, 1.0, 1.0, 1.0]

# Task 3

In [2]:
import datasets

TEST = datasets.load_dataset("nuprl/llm-systems-math-word-problems", split="test")

/Users/nickeldime/Code/LLM Class/CS4973/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
TEST[2]

{'question': 'James decides to run 3 sprints 3 times a week.  He runs 60 meters each sprint.  How many total meters does he run a week?',
 'answer': 540}

In [4]:
print(Homework1.accuracy_zero_shot(TEST))

  0%|          | 0/5 [00:00<?, ?it/s]

[0.14, 0.14, 0.18, 0.16, 0.14]


# Task 4

In [5]:
Homework1.accuracy_few_shot(TEST)

  0%|          | 0/5 [00:00<?, ?it/s]

[0.18, 0.16, 0.18, 0.16, 0.18]

# Task 5

In [3]:
Homework1.accuracy_cot(TEST)

  0%|          | 0/5 [00:00<?, ?it/s]

[0.6, 0.6, 0.6, 0.6, 0.6]

# Task 6

In [3]:
Homework1.accuracy_pal(TEST)

100%|██████████| 5/5 [05:50<00:00, 70.06s/it]


[0.6, 0.62, 0.62, 0.62, 0.62]